# Searching or questioning all the sources
In this notebook, we create a chain that can ask all the different sources a question. Using functions we can choose the right source. We use the language model to make the selection.
- Product search: use the VectorStore retriever making use of Amazon OpenSearch Service
- Question: use the VectorStore QA retriever to use content from the vector store and OpenAI to write an asnwer
- Find Opening times: Use a normal query against Amazon OpenSearch Service

## Initialise the OpenSearch connection 
Notice that we do not provide a default alias. We need to provide the name of the index or alias on each request.

In [ ]:


from retriever import find_auth_opensearch, OpenSearchClient

config = find_auth_opensearch()
client = OpenSearchClient(config)

if client.ping():
    print("We have a connection to the Amazon OpenSearch Cluster")
else:
    print("ERROR: no connection to the Amazon OpenSearch Cluster")

## Initialise the product search

In [ ]:
import os

from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from opensearchpy import RequestsHttpConnection
from dotenv import load_dotenv

load_dotenv()

vector_store_products = OpenSearchVectorSearch(
    index_name="sg-products",
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    opensearch_url=f"https://{config['host']}:{config['port']}",
    use_ssl=True,
    verify_certs=True,
    http_auth=config["auth"],
    connection_class=RequestsHttpConnection
)


def execute_product_search(query: str) -> dict:
    """Useful for finding products related to the search terms that are provided.

    Args:
        query: Query to search products for
    """
    found_docs = vector_store_products.similarity_search_with_score(query=query, 
                                                                    text_field="title",
                                                                    vector_field="title_vector")
    results = []
    for doc, _score in found_docs:
        results.append({"title": doc.page_content, "score": _score, "image_name": doc.metadata["image_name"]})

    return {
        "tool": "product_search",
        "result": results
    }

In [ ]:
print(execute_product_search(query="The lord of the rings"))

## Initialise the content search

In [ ]:
vector_store = OpenSearchVectorSearch(
    index_name="sg-content",
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    opensearch_url=f"https://{config['host']}:{config['port']}",
    use_ssl=True,
    verify_certs=True,
    http_auth=config["auth"],
    connection_class=RequestsHttpConnection
)

from langchain.chains import RetrievalQA
from langchain import PromptTemplate, OpenAI

prompt_template = """Use the context to answer the question. If you don't know the 
    answer, just say that you don't know, don't make up an answer.

    {context}

    Question: {question}:"""

custom_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": custom_prompt}
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs=chain_type_kwargs
)


def execute_content_search(query: str) -> dict:
    """Useful for obtaining answers to questions about help for the website, your account and sustainability.

    Args:
        query: Query to use as input for the content search
    """
    return {
        "tool": "content_search",
        "result": chain.run(query)
    }

In [ ]:
print(execute_content_search("What is your return policy"))

In [ ]:
def execute_opening_hours(query: str) -> dict:
    """Useful for obtaining opening hours of a store in city that is provided.

    Args:
        query: Name of the city to search for
    """
    body = {
        "query": {
            "match": {
                "city": query
            }
        }
    }
    search_result = client.search(body=body, size=1, index_name="sg-stores")
    result = []
    if search_result["hits"]["total"]["value"] > 0:
        result = search_result["hits"]["hits"][0]["_source"]
        
    
    return {
        "tool": "opening_hours",
        "result": result
    }

In [ ]:
print(execute_opening_hours("pijnacker"))

## Initialise the function call chain using the previous functions

In [ ]:
from langchain.chains.openai_functions import create_openai_fn_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


def init_fn_chain():
    llm = ChatOpenAI(model="gpt-4-0613", temperature=0, openai_api_key=os.getenv('OPEN_AI_API_KEY'))
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a system to search for answers on a provided question."),
            ("human", "Make calls to the relevant function to using the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

    return create_openai_fn_chain(
        functions=[execute_opening_hours, execute_content_search, execute_product_search],
        llm=llm,
        prompt=prompt,
        verbose=True
    )

function_chain = init_fn_chain()

functions_map = {
    'execute_opening_hours': execute_opening_hours,
    'execute_content_search': execute_content_search,
    'execute_product_search': execute_product_search
}


In [ ]:
import langchain
langchain.debug = False
# message = "lord of the rings"
# message = "Do you use reusable packaging"
message = "How can I reset my password"
# message = "Do you have something from Lord of the rings?"
# message = "openinghours pijnacker"
# message = "openingstijden pijnacker"
# response = agent_executor.run(message)

chain_response = function_chain.run(message)
print(chain_response)
function_name = chain_response["name"]
args = chain_response["arguments"]

if function_name not in functions_map:
    print("Could not decide which content search to perform.")
else:
    print(functions_map[function_name](**args))

